Intro for all models; this'll be the same for everyone; post any changes so that everyone can do it

In [17]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\tweek\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,classification_report,f1_score,roc_auc_score
import tensorflow as tf
from tensorflow import keras
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

from scipy.stats import norm

In [3]:
df=pd.read_excel('ModelingDataset.xlsx')

In [4]:
#this should've been done in the data cleaning step, Nehal can you add this to our later reports/ppt as part of data cleaning?
df=df[((df['term_category'].isna())&(df['employee_status']=='Active'))|((df['term_category']=='Voluntary Termination')&(df['employee_status']=='Terminated'))]
#output var creation
df['employee_status']=df['employee_status'].apply(lambda x: 1 if x=='Terminated' else 0)

KeyError: 'term_category'

In [ ]:
#these vars can directly predict output variable; need to drop them for sure
#work structure has a very high correlation with the output, hence dropped. base_pay_mid_point has high correlation with Salary, hence dropped
df.drop(['term_category','termination_reason','termination_date','last_promotion_date','work_structure','base_pay_mid_point'],axis=1,inplace=True)

In [ ]:
#Changing date columns to years
#Should've been in data cleaning too 

for d in [col for col in df.columns if 'date' in col]:
    df[d]=df[d].apply(lambda x: str(x.year))

#Binning data based on quartiles
#dropping old columns here
for col in [col for col in df.columns if 'date' in col]:
    print(col)
    # Fill null values
    df[col].fillna('NA', inplace=True)

    # Get value counts of each category
    value_counts = df[col].value_counts()

    # Define bins based on quartiles
    bins = [0, value_counts.quantile(0.25), value_counts.quantile(0.5), value_counts.quantile(0.75), np.inf]
    labels = ['Q1', 'Q2', 'Q3', 'Q4']

    # Creating new variable name for the binned data
    new_var = col + '_q_binned'

    # Bin categories based on quartiles
    df[new_var] = pd.cut(df[col].map(value_counts), bins=bins, labels=labels, include_lowest=True)
    
    #dropping old date columns
    #TODO - need to map quartiles to old years to understand 
    df.drop(col,axis=1,inplace=True)
    print(new_var)

In [ ]:
# More data cleaning steps; need to add as having been done before in data cleaning report
df['days_since_promotion'][df['days_since_promotion'].isna()]=-1
df['marital_status'][df['marital_status'].isna()]='NA'

In [ ]:
#Creating dummies and replacing them in the original data


num_cols=[]
cat_cols=[]
for x in df.columns:  
    if df[x].dtype=='object':
        cat_cols.append(x)
    elif len(set(df[x]))<10:
        cat_cols.append(x)
    else:
        num_cols.append(x)
        
print(num_cols)
print(cat_cols)


#Getting dummies for variables, drop old ones
for var in cat_cols:
    if var not in ['employee_status','cost_to_replace_factor']:
        dummies = pd.get_dummies(df[var], prefix=var, drop_first=True)
        df = pd.concat([df, dummies], axis=1)
        df.drop(var,axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
# calculate correlation matrix
corr_matrix = df.corr()

# display correlation matrix with just the correlations for the target variable
# sort correlation matrix by correlations with the target variable, in descending order
corr_matrix_sorted = corr_matrix['employee_status'].sort_values(ascending=False)

# display sorted correlation matrix
print(corr_matrix_sorted)

In [ ]:
# Set style and color palette
sns.set(style="white")
sns.set_palette("coolwarm")

# Create heatmap with correlation matrix
fig, ax = plt.subplots(figsize=(10,10))
# plot correlation matrix as a heatmap
sns.heatmap(corr_matrix, cmap="coolwarm")
# show the plot
plt.show()

In [7]:
df.to_excel('ModelingDataset.xlsx')


KeyboardInterrupt



Modelling (use your code here)

In [8]:
# set random seed
random_seed = 42

In [9]:
import os

In [5]:
df=pd.read_excel('ModelingDataset.xlsx')

In [6]:
df.shape

(20321, 73)

In [10]:
# separate features and target variable
X = df.drop('employee_status', axis=1)
y = df['employee_status']

# split data into train and test sets, stratifying by the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed, stratify=y)

# check the ratio of binary variable in train and test set
print("Ratio of binary variable in train set: ", y_train.sum() / len(y_train))
print("Ratio of binary variable in test set: ", y_test.sum() / len(y_test))


Ratio of binary variable in train set:  0.27165354330708663
Ratio of binary variable in test set:  0.2716089880268985


In [11]:
target_column = ['employee_status'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()

In [12]:
X = df[predictors].values
y = df[target_column].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=random_seed, stratify = y)
print(X_train.shape); print(X_test.shape)

(14224, 72)
(6097, 72)


In [ ]:
from sklearn.neural_network import MLPClassifier
for i in [6,8,10,12,16,20,24]:
    mlp = MLPClassifier(hidden_layer_sizes=(i,i), activation='relu', solver='adam', max_iter=500, random_state=random_seed)
    mlp.fit(X_train, y_train.ravel())
    predict_train = mlp.predict(X_train)
    predict_test = mlp.predict(X_test)
    print("Current nodes is: "+ str(i))
    print(confusion_matrix(y_test,predict_test))
    print(classification_report(y_test,predict_test))

In [13]:
mlp = MLPClassifier(hidden_layer_sizes=(24,24,24), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train, y_train.ravel())
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[4029  412]
 [ 669  987]]
              precision    recall  f1-score   support

           0       0.86      0.91      0.88      4441
           1       0.71      0.60      0.65      1656

    accuracy                           0.82      6097
   macro avg       0.78      0.75      0.76      6097
weighted avg       0.82      0.82      0.82      6097



In [14]:
df2=pd.read_excel('ModelingDataset-LastYear.xlsx')

In [15]:
df3=pd.read_excel('ModelingDataset-LastYear-NoNormalization.xlsx')

In [16]:
df2 = df2.drop(columns=['Unnamed: 0'])
print(df2.shape)

(15770, 73)


In [17]:
# separate features and target variable
X2 = df2.drop('employee_status', axis=1)
y2 = df2['employee_status']

# split data into train and test sets, stratifying by the target variable
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=random_seed, stratify=y2)

In [18]:
predict_test = mlp.predict(X2)
print(confusion_matrix(y2,predict_test))
print(classification_report(y2,predict_test))

[[13532  1269]
 [  922    47]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     14801
           1       0.04      0.05      0.04       969

    accuracy                           0.86     15770
   macro avg       0.49      0.48      0.48     15770
weighted avg       0.88      0.86      0.87     15770



c:\users\tweek\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [19]:
predictArray = mlp.predict(X2)

c:\users\tweek\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [20]:
df3['predict'] = predictArray

In [24]:
compaRatios = np.arange(.80,1.21,.02)
compaRatiosNormal = []
mu = df2['compa_ratio'].mean()
sigma = df2['compa_ratio'].std()
for i in compaRatios:
    print(norm.cdf((i- mu) / sigma))
    compaRatiosNormal.append(norm.cdf((i- mu) / sigma))

0.8300553774000394
0.8479992279320463
0.8646378303330915
0.8799830303464462
0.8940591442294794
0.9069015828532874
0.9185553606853488
0.9290735377756345
0.9385156407225814
0.9469461049962336
0.9544327762229544
0.9610455023979654
0.9668548428059722
0.9719309130056291
0.9763423788569456
0.9801556064935949
0.9834339695726023
0.9862373102316879
0.9886215460586676
0.9906384120875316
0.9923353243961844


In [39]:
#Getting numbers
#Amount of Replacement Cost In the last year | 105095412.54429126
print(df3['replacement_cost'][df3['predict']==1].sum())
#Salaries paid in the last year | 812621033.0736899
print(df3['salary'][df3['predict']==0].sum())
print
#Amount of voluntary churn in the last year | 1683
print(df3['predict'][df3['predict']==1].shape)
#Average Compa Ratio of churned employees | 0.9664218657159834
print(df3['compa_ratio'][df3['predict']==1].sum()/df3['predict'][df3['predict']==1].sum())

281392431.6367012
666814906.9362501
(4790,)
0.9388299582463464


In [67]:
#X_train.shape

(14224, 72)

Metrics

In [42]:
# calculate evaluation metrics
accuracy = accuracy_score(y_test, predict_test)
precision = precision_score(y_test, predict_test)
recall = recall_score(y_test, predict_test)

# display evaluation metrics and confusion matrix
print("")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

cm = confusion_matrix(y_test, predict_test)
print("Confusion matrix:")
print(cm)


Accuracy: 0.8187633262260128
Precision: 0.7177865612648221
Recall: 0.5483091787439613
Confusion matrix:
[[4084  357]
 [ 748  908]]


In [43]:
f1 = f1_score(y_test, predict_test)
auc_score = roc_auc_score(y_test, predict_test)

print("F1 score:", f1)
print("AUC score:", auc_score)

F1 score: 0.6217048955837041
AUC score: 0.7339609392931696
